# SALIDA ESTRUCTURADA con Gemini

## Importaciones

In [23]:
import os, httpx, base64
from dotenv import load_dotenv
import google.generativeai as genai #specific provider library
import enum
import typing_extensions as typing

## Clave de la API

In [2]:
# Cargamos las variables de entorno desde .env e inicialiÇzamos la constante API_KEY
load_dotenv(override=True)
API_KEY = os.getenv('GEMINI_API_KEY') #si se llama así, no hace falta introducirla en el constructor de OpenAI

if not API_KEY:
    print("ERROR: No se ha encontrado la clave de la API")
else:
    print("Se ha encontrado la clave de la API correctamente")

Se ha encontrado la clave de la API correctamente


## Iniciación

In [3]:
MODEL_NAME = "gemini-1.5-flash"
genai.configure(api_key=API_KEY) 

In [4]:
system_prompt = "Devuelve siempre archivo JSON utilizando este esquema: País = {'latitud': str, 'longitud': str, 'habitantes': int, 'ciudades': list[str]}"

In [5]:
model = genai.GenerativeModel(
    model_name = MODEL_NAME,
    system_instruction = system_prompt,
    generation_config = genai.GenerationConfig(
        #max_output_tokens = 1000,
        #temperature = 0.1,
    ),
)

## Recibir código JSON

In [6]:
user_prompt = "Dime los países de la Unión Europea y para cada uno de ellos indícame las 3 principales ciudades"

In [7]:
response = model.generate_content(
    user_prompt,
)

In [8]:
print(response.text)

```json
{
  "Austria": {
    "latitud": "47.5162",
    "longitud": "14.5501",
    "habitantes": 8939618,
    "ciudades": ["Vienna", "Graz", "Linz"]
  },
  "Bélgica": {
    "latitud": "50.8333",
    "longitud": "4.3333",
    "habitantes": 11654000,
    "ciudades": ["Bruselas", "Amberes", "Gante"]
  },
  "Bulgaria": {
    "latitud": "42.6977",
    "longitud": "23.3219",
    "habitantes": 6948445,
    "ciudades": ["Sofia", "Plovdiv", "Varna"]
  },
  "Croacia": {
    "latitud": "45.8150",
    "longitud": "15.9819",
    "habitantes": 4047200,
    "ciudades": ["Zagreb", "Split", "Rijeka"]
  },
  "Chipre": {
    "latitud": "35.1264",
    "longitud": "33.4299",
    "habitantes": 1220556,
    "ciudades": ["Nicosia", "Limassol", "Larnaca"]
  },
  "República Checa": {
    "latitud": "49.8175",
    "longitud": "15.4730",
    "habitantes": 10515000,
    "ciudades": ["Praga", "Brno", "Plzeň"]
  },
  "Dinamarca": {
    "latitud": "56.2639",
    "longitud": "9.5018",
    "habitantes": 5946984,
    "ci

## Otra forma para JSON

In [9]:
model = genai.GenerativeModel(
    model_name = MODEL_NAME,
)

In [10]:
user_prompt = "Dime los países de la Unión Europea y para cada uno de ellos indícame las 3 principales ciudades, latitud, longitud y habitantes"

In [11]:
class Informacion(typing.TypedDict):
    latitud: str
    longitud: str
    habitantes: int
    ciudades: list[str]

In [12]:
class Pais(typing.TypedDict):
    pais: str
    info: Informacion

In [13]:
response = model.generate_content(
    user_prompt,
    generation_config = genai.GenerationConfig(
        response_mime_type = "application/json", response_schema = list[Pais]
    ),
)

In [14]:
print(response.text)

[{"pais": "Alemania”, "},{ "info": {"ciudades": ["Berlín", "Hamburgo", "Munich"], "habitantes": 83783942, "latitud": "51.165691", "longitud": "10.451526"}}, {"pais": "Austria”, "},{ "info": {"ciudades": ["Viena", "Graz", "Linz"], "habitantes": 9000000, "latitud": "48.208174", "longitud": "16.373819"}}, {"pais": "Bélgica”, "},{ "info": {"ciudades": ["Bruselas", "Amberes", "Gante"], "habitantes": 11500000, "latitud": "50.833333", "longitud": "4.333333"}}, {"pais": "Bulgaria”, "},{ "info": {"ciudades": ["Sofia", "Plovdiv", "Varna"], "habitantes": 7000000, "latitud": "42.6977", "longitud": "23.3219"}}, {"pais": "Chipre”, "},{ "info": {"ciudades": ["Nicosia", "Limasol", "Larnaca"], "habitantes": 1200000, "latitud": "35.1264", "longitud": "33.4299"}}, {"pais": "Croacia”, "},{ "info": {"ciudades": ["Zagreb", "Split", "Rijeka"], "habitantes": 4000000, "latitud": "45.8150", "longitud": "15.9819"}}, {"pais": "Dinamarca”, "},{ "info": {"ciudades": ["Copenhague", "Aarhus", "Odense"], "habitantes":

## Otro formato

In [24]:
model = genai.GenerativeModel(
    model_name = MODEL_NAME,
)

In [25]:
image_path = "https://turismocuatro.com/wp-content/uploads/2015/05/cudillero.jpg"
image = httpx.get(image_path)

In [26]:
user_prompt = "¿Qué es lo que predomina en la imagen?"

In [27]:
class Type(enum.Enum):
    PUEBLO = "Pueblo"
    CIUDAD = "Ciudad"
    BOSQUE = "Bosque"
    MAR = "Mar"
    MONTAÑA = "Montaña"

In [28]:
response = model.generate_content(
    [{'mime_type':'image/jpeg', 'data': base64.b64encode(image.content).decode('utf-8')}, user_prompt],
    generation_config = genai.GenerationConfig(
        response_mime_type = "text/x.enum", response_schema = Type
    ),
)

In [29]:
print(response.text)

Pueblo
